In [1]:
import torch
import numpy as np
from phi.opt_utils import get_test_prompts, get_last_token_activations_single, load_model_and_tokenizer
from utils.data import format_prompts
from utils.load_file_paths import load_file_paths
import os
import random
import json
import gc
import time
from constants import PROJECT_ROOT, LAYER_MAP
import torch.nn as nn
import copy

np.set_printoptions(suppress=True, linewidth=10000)
torch.set_printoptions(sci_mode=False, linewidth=100000, threshold=float('inf'))

/home/40456997@eeecs.qub.ac.uk/PycharmProjects/TaskDriftTest/venv/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
model_name = 'phi3'

In [3]:
model_path = f'{PROJECT_ROOT}/loaded_models/{model_name}'

In [4]:
torch.cuda.empty_cache()

In [5]:
model, tokenizer = load_model_and_tokenizer(model_path, torch_dtype=torch.float32 if model_name == 'llama3_8b' else torch.bfloat16)

print(model.dtype)

device = model.get_input_embeddings().weight.device

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

torch.bfloat16


In [6]:
test_prompts = get_test_prompts()
print(len(test_prompts))

31134


In [7]:

class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.linear(x).squeeze(-1)


In [8]:
linear_models = {}

layers = []

model_type = 'suffix'  #  Either pgd or suffix

for (dirpath, dir_names, filenames) in os.walk(f'{PROJECT_ROOT}/adv_trained_linear_probes_{model_type}/{model_name}'):
    layers = [int(dir_name) for dir_name in dir_names]
    break

layers.sort()

for i in layers:
    linear_models[i] = LogisticRegression(input_dim=3072)
    linear_models[i].load_state_dict(torch.load(os.path.join(PROJECT_ROOT, f'adv_trained_linear_probes_{model_type}', model_name, str(i), 'model.pt')))
    linear_models[i].eval()


In [9]:

# Store these to avoid loading the activation files repeatedly, which is an expensive operation

previous_activation_filepath = ''
activations = None

def get_primary_activation(index, model, layer, subset):

    global previous_activation_filepath, activations

    index_in_file = index - int(index / 1000) * 1000

    if subset == 'test':
        filepaths = load_file_paths(f'{PROJECT_ROOT}/data_files/test_poisoned_files_{model}.txt')
    else:
        filepaths = load_file_paths(f'{PROJECT_ROOT}/data_files/train_files_{model}.txt')

    activation_file_index_in_list = 0

    for idx, filepath in enumerate(filepaths):
        if filepath.count(f'_{int(index / 1000) * 1000}_{(int(index / 1000) + 1) * 1000}_') == 1:
            activation_file_index_in_list = idx
            break

    current_activation_filepath = f'/home/40456997@eeecs.qub.ac.uk/Reduced Activation/{model}/{subset}/{filepaths[activation_file_index_in_list]}'

    if previous_activation_filepath != current_activation_filepath:
        activations = torch.load(current_activation_filepath)

    previous_activation_filepath = current_activation_filepath

    return activations[0][index_in_file][LAYER_MAP[layer]]


In [10]:
def check_task_drift(prompt_index, hidden_states):

    labels = []
    probs = []

    for num_layer, linear_model in linear_models.items():
        primary_activation = get_primary_activation(prompt_index, model_name, num_layer, subset='test')
        poisoned_activation = hidden_states[num_layer][:, -1].cpu()

        delta = (poisoned_activation - primary_activation).to(torch.float32)

        with torch.no_grad():
            logits = linear_model(delta)
            prob = torch.sigmoid(logits)
            label = (prob >= .5).long()

            labels.append(label.item())
            probs.append([1 - prob.item(), prob.item()])

    return labels, probs


In [11]:
# adv_suffix = r"""comuna cidade comuna$\ $\{6 +\frac revision Guillaume,..., Riemann\,\mapsto$\textbf MRlabel converges',"""

In [12]:
# print(len(tokenizer(adv_suffix).input_ids))

In [13]:
def format_probs(probs):
    formatted_probs = []
    for prob_pair in probs:
        formatted_pair = [f"{p:.8f}" for p in prob_pair]
        formatted_probs.append(f"[{formatted_pair[0]}, {formatted_pair[1]}]")
    probs_str = "[" + ", ".join(formatted_probs) + "]"

    return probs_str

In [14]:

result_path = f'{PROJECT_ROOT}/test_results/{model_name}_result_adv_trained_models.json'

if os.path.exists(result_path):
    result = json.load(open(result_path, 'r'))
else:

    result = {
        'Result list': [

        ]
    }


In [ ]:

optimisation_result_path = f'{PROJECT_ROOT}/opt_results/{model_name}_optimisation_result.json'

adv_suffixes = []

if model_type == 'pgd':
    if os.path.exists(optimisation_result_path):
        optimisation_result = json.load(open(optimisation_result_path, 'r'))
        for opt_result in optimisation_result['Result list'][len(result['Result list']):]:
            adv_suffixes.append(opt_result['Result']['Iteration log'][-1]['suffix'])
else:
    test_suffixes = json.load(open(f'{PROJECT_ROOT}/phi3_test_suffix_list.json', 'r'))
    adv_suffixes = test_suffixes['Suffix list']

for adv_suf in adv_suffixes:
    print(adv_suf)



In [ ]:

start_time = time.time()

start_prompt = 0
total_prompts = 31134

for adv_suffix in adv_suffixes:

    prompt_indices = random.sample(range(total_prompts), 12000)

    # Indices must be sorted to avoid loading same activation file multiple times
    prompt_indices.sort()

    result_dict = {
        "Suffix": adv_suffix,
        "Prompt indices": prompt_indices,
        "Attack result list": [],
        "Total number of prompts correctly classified by a specific number of classifiers": {
            "Without suffix": {str(key): 0 for key in range(len(layers) + 1)},
            "With suffix": {str(key): 0 for key in range(len(layers) + 1)}
        },
        "Layerwise correct classification": {
            "Without suffix": {str(key): 0 for key in layers},
            "With suffix": {str(key): 0 for key in layers}
        }
    }

    cnt_correct_classification_without_suffix = result_dict["Total number of prompts correctly classified by a specific number of classifiers"]["Without suffix"]
    cnt_correct_classification_with_suffix = result_dict["Total number of prompts correctly classified by a specific number of classifiers"]["With suffix"]

    layerwise_correct_classification_without_suffix = result_dict["Layerwise correct classification"]["Without suffix"]
    layerwise_correct_classification_with_suffix = result_dict["Layerwise correct classification"]["With suffix"]


    print(f"Adv suffix: {adv_suffix}")

    for prompt_index in prompt_indices:
        gc.collect()

        prompt_without_adv_suffix = copy.deepcopy(test_prompts[prompt_index])

        primary, primary_clean, primary_poisoned = (
            format_prompts([prompt_without_adv_suffix], True)
        )

        outputs = get_last_token_activations_single(primary_poisoned[0], tokenizer, model)
        hidden_states = outputs['hidden_states'][1:]

        labels_without_adv_suffix, probs_without_adv_suffix = check_task_drift(prompt_index, hidden_states)

        cnt_correct_classification_without_suffix[str(labels_without_adv_suffix.count(1))] += 1

        for i, num_layer in enumerate(layers):
            layerwise_correct_classification_without_suffix[str(num_layer)] += labels_without_adv_suffix[i]

        result_dict["Attack result list"].append(
            {
                "Without suffix": {
                    "labels": labels_without_adv_suffix,
                    "probs": probs_without_adv_suffix,
                },
            }
        )

        # ----------------------------------------------------------------------------------------------

        prompt_with_adv_suffix = copy.deepcopy(test_prompts[prompt_index])

        prompt_with_adv_suffix['final_text_paragraph'] = prompt_with_adv_suffix['final_text_paragraph'] + " " + adv_suffix

        primary, primary_clean, primary_poisoned = (
            format_prompts([prompt_with_adv_suffix], True)
        )

        outputs = get_last_token_activations_single(primary_poisoned[0], tokenizer, model)
        hidden_states = outputs['hidden_states'][1:]

        labels_with_adv_suffix, probs_with_adv_suffix = check_task_drift(prompt_index, hidden_states)

        cnt_correct_classification_with_suffix[str(labels_with_adv_suffix.count(1))] += 1

        for i, num_layer in enumerate(layers):
            layerwise_correct_classification_with_suffix[str(num_layer)] += labels_with_adv_suffix[i]

        result_dict["Attack result list"][-1]["With suffix"] = {
                    "labels": labels_with_adv_suffix,
                    "probs": probs_with_adv_suffix,
                }

        print(f"Prompt index: {prompt_index}")
        print(f"Without suffix:    labels: {labels_without_adv_suffix}  probs: {format_probs(probs_without_adv_suffix)}")
        print(f"With suffix:       labels: {labels_with_adv_suffix}  probs: {format_probs(probs_with_adv_suffix)}\n")


    cur_time = time.time()
    print(f"Total elapsed time: {cur_time - start_time} seconds\n")

    print("Total number of prompts correctly classified by a specific number of classifiers")
    print(f"Without suffix: {cnt_correct_classification_without_suffix}")
    print(f"With suffix: {cnt_correct_classification_with_suffix}\n")

    print(f"Layerwise correct classification without suffix: {layerwise_correct_classification_without_suffix}")
    print(f"Layerwise correct classification with suffix: {layerwise_correct_classification_with_suffix}\n")

    print("---------------------------------------------------------------------------------------------------------------------")


    result['Result list'].append(result_dict)

    with open(result_path, 'w') as f:
        json.dump(result, f, indent=4)
